# Model Building Setup for Safety Models using RdKit descriptors and ChEMBL31 data only

# Setup

In [1]:
# input
model_dir = "AHR_PEC50"
data_file_name = "./AHR_PEC50_train.tsv"
# output
scalar_file_name = "./scalar_AHR_PEC50.pkl"
svm_model_file = "./AHR_PEC50_svm.pkl"
xgb_model_file = "./AHR_PEC50_xgb.pkl"

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
import numpy as np
import pandas as pd
import joblib
from sklearn.svm import SVC
import xgboost
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef, confusion_matrix, balanced_accuracy_score, recall_score
from utils import rdkit_fpconvert_numpy, rdkit_get_physchem_descr

seed = 1234

# Get Mols & Data then calculate descriptors

In [3]:
df = pd.read_csv(data_file_name, sep="\t")
df.head()

,parentised_smiles,ID,class_label
0,Brc1cc(Br)c2Oc3c(Br)cc(Br)cc3Oc2c1,chembl_15460,POSITIVE
1,Brc1cc(Br)c2Oc3cc(Br)c(Br)cc3Oc2c1,chembl_15569,POSITIVE
2,Brc1cc2Oc3c(Br)cc(Br)c(Br)c3Oc2cc1Br,chembl_15664,POSITIVE
3,Brc1cc2Oc3cc(Br)c(Br)c(Br)c3Oc2cc1Br,chembl_15556,POSITIVE
4,Brc1cc2Oc3cc(Br)c(Br)cc3Oc2cc1Br,chembl_15580,POSITIVE


In [4]:
df.value_counts("class_label")

class_label
POSITIVE    130
NEGATIVE     79
dtype: int64

## Preparing descriptors and responses now

In [5]:
y_str = df.class_label.tolist() # convert str to list
y_int = pd.get_dummies(y_str) # one hot encode the str labels
y_new = y_int["POSITIVE"] # set new y to one hot encoded POSITIVE label column 
df['class_label_binary'] = y_new # set new encoded label into dataframe
y = df.class_label_binary.to_list()

In [6]:
df.head(3)

,parentised_smiles,ID,class_label,class_label_binary
0,Brc1cc(Br)c2Oc3c(Br)cc(Br)cc3Oc2c1,chembl_15460,POSITIVE,1
1,Brc1cc(Br)c2Oc3cc(Br)c(Br)cc3Oc2c1,chembl_15569,POSITIVE,1
2,Brc1cc2Oc3c(Br)cc(Br)c(Br)c3Oc2cc1Br,chembl_15664,POSITIVE,1


In [7]:
mols = [Chem.MolFromSmiles(smi) for smi in df.parentised_smiles]
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]
# generate binary Morgan fingerprint with radius 2 with physchem as numpy array
x = rdkit_fpconvert_numpy(fp)
x = np.concatenate((x, rdkit_get_physchem_descr(mols)), axis=1)

## Scale Data and Calculate Splits

In [8]:
# randomly select 20% of compounds as test set but with stratified selection
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed, stratify=y)

### Investigate size of splits to confirm

In [9]:
len(y_tr), len(x_tr), len(x_ts), len(y_ts)

(167, 167, 42, 42)

In [10]:
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [11]:
# save scalar
joblib.dump(scale, 
            scalar_file_name, 
            compress=3)

['./scalar_AHR_PEC50.pkl']

In [12]:
# cross validation splits
cv = StratifiedKFold(n_splits=5)

# SVM Model

Search for optimal tuning parameters and build the model

In [13]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [14]:
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [15]:
# run model building
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [16]:
svm.best_score_

0.6285204991087345

In [17]:
svm.best_params_

{'C': 100, 'gamma': 1e-05}

In [18]:
# save model
joblib.dump(svm, 
            svm_model_file,
            compress=3)

['./AHR_PEC50_svm.pkl']

# Test Set Validation

In [19]:
# predict for the test set compounds
pred_svm = svm.predict(x_ts)

In [20]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_svm))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_ts, pred_svm))
print("MCC = ", matthews_corrcoef(y_ts, pred_svm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_svm))
print("Recall = ", recall_score(y_ts, pred_svm))
print("Performance reported as average of matthews and recall is", (((matthews_corrcoef(y_ts, pred_svm))+(recall_score(y_ts, pred_svm)))/2))

Accuracy =  0.7380952380952381
Balanced Accuracy Score =  0.6802884615384616
MCC =  0.4267429637493243
Kappa =  0.39370078740157477
Recall =  0.9230769230769231
Performance reported as average of matthews and recall is 0.6749099434131237


In [21]:
# confusion matrix
confusion_matrix(y_ts, pred_svm)

array([[ 7,  9],
       [ 2, 24]])

In [22]:
tn, fp, fn, tp = confusion_matrix(y_ts, pred_svm).ravel()
tn, fp, fn, tp

(7, 9, 2, 24)

# XGB Model

In [23]:
# create grid search dictionary
param_grid = {"max_depth": [i for i in range(2, 8)],
              "n_estimators": [i for i in [10, 30, 50, 100]],
              "learning_rate": [i for i in [0.01, 0.05, 0.1]]}

In [24]:
# setup model building
xgb = GridSearchCV(xgboost.XGBClassifier(base_score=0.5), param_grid, n_jobs=2, cv=cv, verbose=1)

In [25]:
# run model building
xgb.fit(x_tr, y_tr)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=0.5, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_t...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan

In [26]:
xgb.best_score_

0.6768270944741535

In [27]:
xgb.best_params_

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 10}

In [28]:
# save model
joblib.dump(xgb, 
            xgb_model_file,
            compress=3)

['./AHR_PEC50_xgb.pkl']

## Test Set Validation

In [29]:
# predict for the test set compounds
pred_xgb = xgb.predict(x_ts)

In [30]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_xgb))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_ts, pred_xgb))
print("MCC = ", matthews_corrcoef(y_ts, pred_xgb))
print("Kappa = ", cohen_kappa_score(y_ts, pred_xgb))
print("Recall = ", recall_score(y_ts, pred_xgb))

Accuracy =  0.6666666666666666
Balanced Accuracy Score =  0.6105769230769231
MCC =  0.25215418055077093
Kappa =  0.23834196891191706
Recall =  0.8461538461538461


In [31]:
# confusion matrix
confusion_matrix(y_ts, pred_xgb)

array([[ 6, 10],
       [ 4, 22]])

In [32]:
tn, fp, fn, tp = confusion_matrix(y_ts, pred_xgb).ravel()
tn, fp, fn, tp

(6, 10, 4, 22)

# Pipeline for making new predictions

In [37]:
predictor_model = joblib.load(xgb_model_file)
smi_file = "../smi_file.txt"
pred_example = pd.read_csv('example_drugs.txt')
pred_example.head(3)

,smiles,id
0,[H][C@]12C[C@@H](C(=O)N(CCCN(C)C)C(=O)NCC)CN(C...,Cabergoline
1,CN(C)CCCN1c2ccccc2Sc2ccc(Cl)cc21,Chlorpromazine
2,Fc1ccc(cc1)Cn2c5ccccc5nc2NC4CCN(CCc3ccc(OC)cc3...,Astemiszole


In [39]:
pred_mols = [Chem.MolFromSmiles(smi) for smi in pred_example.smiles]
pred_fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in pred_mols]
pred_x = rdkit_fpconvert_numpy(pred_fp)

pred_x = np.concatenate((pred_x, rdkit_get_physchem_descr(pred_mols)), axis=1)
pred_x = scale.transform(pred_x)

In [40]:
pred_best_model = predictor_model.predict(pred_x)
pred_best_model

array([1, 1, 1, 0, 1])

A prediction of 1 suggests a positive class label prediction and 0 a negative prediction